In [2]:
import numpy as np
import tensorflow as tf
from tensorflow import keras
import matplotlib.pyplot as plt
import sqlite3
import pymatching
import pandas as pd

In [3]:
# Initialize Dataset
conn_test = sqlite3.connect("../db/surf17_test.db")
conn_train = sqlite3.connect("../db/surf17_train.db")
conn_validation = sqlite3.connect("../db/surf17_validation.db")



In [3]:
model = models.sequential([
    layers.Flatten(input_shape = (X_train))
])

In [ ]:
def compute_logical_error_rate(model, X, y_true, parities):
    predictions = (model.predict(X) > 0.5).astype(bool)  # Threshold predictions
    errors = 0
    for pred, true, parity in zip(predictions, y_true, parities):
        # Check if predicted err_signal leads to correct parity (simplified)
        if not np.array_equal(pred, true):  # If mismatch, count as error
            errors += 1
    return errors / len(X)

parities_test = [d[5] for d in test_data]
ler_nn = compute_logical_error_rate(model, X_test, y_test, parities_test)
print(f"NN Logical Error Rate: {ler_nn}")

In [ ]:
# Define the parity check matrix for surface code (distance 3, simplified)
# For a distance 3 surface code, the PCM has rows for X and Z stabilizers
# This is a basic example; adjust for your exact layout from circuit_model.py
H = np.array([
    [1, 0, 1, 0, 1, 0, 0, 0, 0],  # Example row for X stabilizer
    # Add more rows for full PCM (9 data qubits for dist=3)
    # ... (full H matrix here, based on surface code geometry)
])
# For simplicity, assume H is predefined; in practice, generate from SurfaceCode class

matcher = pymatching.Matching(H)

def decode_with_mwpm(syndromes, distance=3):
    # syndromes is a 1D array of defect positions
    # Convert to pymatching format
    syndrome_array = np.array(syndromes, dtype=int)
    predicted_errors = matcher.decode(syndrome_array)
    return predicted_errors  # Binary array of predicted flips

# Evaluate MWPM on test data
def compute_logical_error_rate_mwpm(data, distance=3):
    errors = 0
    for d in data:
        syndromes = d[1]  # syndromes array
        true_parity = d[5]  # true parity
        predicted_errors = decode_with_mwpm(syndromes[-1], distance)  # Use final syndrome
        # Compute predicted parity from predicted errors (simplified: parity of error flips)
        pred_parity = np.sum(predicted_errors) % 2
        if pred_parity != true_parity:
            errors += 1
    return errors / len(data)

ler_mwpm = compute_logical_error_rate_mwpm(test_data)
print(f"MWPM Logical Error Rate: {ler_mwpm}")

# Compare
print(f"NN LER: {ler_nn}, MWPM LER: {ler_mwpm}")

In [ ]:
# Test pull